In [1]:
import pandas as pd
import os
import warnings
warnings.filterwarnings('ignore')
data_dst = 'data'

Load previously processed data. 

In [2]:
portfolio = pd.read_json(os.path.join(data_dst, 'portfolio.json'), orient='records', lines=True)
profile = pd.read_csv(os.path.join(data_dst, 'profile_clean.csv'))
response = pd.read_csv(os.path.join(data_dst, 'response.csv'))

## Further process data
Process data saved from last notebook, and drop the original columns not suitable as inputs for a model. 

### portfolio

In [3]:
portfolio

,reward,channels,difficulty,duration,offer_type,id
0,10,"[email, mobile, social]",10,7,bogo,ae264e3637204a6fb9bb56bc8210ddfd
1,10,"[web, email, mobile, social]",10,5,bogo,4d5c57ea9a6940dd891ad53e9dbe8da0
2,0,"[web, email, mobile]",0,4,informational,3f207df678b143eea3cee63160fa8bed
3,5,"[web, email, mobile]",5,7,bogo,9b98b8c7a33c4b65b9aebfe6a799e6d9
4,5,"[web, email]",20,10,discount,0b1e1539f2cc45b7b9fa7c272da2e1d7
5,3,"[web, email, mobile, social]",7,7,discount,2298d6c36e964ae4a3e7e9706d1fb8c2
6,2,"[web, email, mobile, social]",10,10,discount,fafdcd668e3743c1bb461111dcafc2a4
7,0,"[email, mobile, social]",0,3,informational,5a8bc65990b245e5a138643cd4eb9837
8,5,"[web, email, mobile, social]",5,5,bogo,f19421c1d4aa40978ebb69ca19b0e20d
9,2,"[web, email, mobile]",10,7,discount,2906b810c7d4411798c6938adc9daaa5


The 'channels' column contains the four media where the offer is sent. Use one-hot encoding for each one of them. 

In [4]:
for channel in ['web', 'email', 'mobile', 'social']:
    portfolio['channel_%s' % channel] = portfolio['channels'].apply(lambda c: int(channel in c))
portfolio

,reward,channels,difficulty,duration,offer_type,id,channel_web,channel_email,channel_mobile,channel_social
0,10,"[email, mobile, social]",10,7,bogo,ae264e3637204a6fb9bb56bc8210ddfd,0,1,1,1
1,10,"[web, email, mobile, social]",10,5,bogo,4d5c57ea9a6940dd891ad53e9dbe8da0,1,1,1,1
2,0,"[web, email, mobile]",0,4,informational,3f207df678b143eea3cee63160fa8bed,1,1,1,0
3,5,"[web, email, mobile]",5,7,bogo,9b98b8c7a33c4b65b9aebfe6a799e6d9,1,1,1,0
4,5,"[web, email]",20,10,discount,0b1e1539f2cc45b7b9fa7c272da2e1d7,1,1,0,0
5,3,"[web, email, mobile, social]",7,7,discount,2298d6c36e964ae4a3e7e9706d1fb8c2,1,1,1,1
6,2,"[web, email, mobile, social]",10,10,discount,fafdcd668e3743c1bb461111dcafc2a4,1,1,1,1
7,0,"[email, mobile, social]",0,3,informational,5a8bc65990b245e5a138643cd4eb9837,0,1,1,1
8,5,"[web, email, mobile, social]",5,5,bogo,f19421c1d4aa40978ebb69ca19b0e20d,1,1,1,1
9,2,"[web, email, mobile]",10,7,discount,2906b810c7d4411798c6938adc9daaa5,1,1,1,0


Looks like all offers are sent via email, so the 'channel_email' column should also be dropped.

In [5]:
portfolio.drop(columns=['channels', 'channel_email'], inplace=True)
portfolio

,reward,difficulty,duration,offer_type,id,channel_web,channel_mobile,channel_social
0,10,10,7,bogo,ae264e3637204a6fb9bb56bc8210ddfd,0,1,1
1,10,10,5,bogo,4d5c57ea9a6940dd891ad53e9dbe8da0,1,1,1
2,0,0,4,informational,3f207df678b143eea3cee63160fa8bed,1,1,0
3,5,5,7,bogo,9b98b8c7a33c4b65b9aebfe6a799e6d9,1,1,0
4,5,20,10,discount,0b1e1539f2cc45b7b9fa7c272da2e1d7,1,0,0
5,3,7,7,discount,2298d6c36e964ae4a3e7e9706d1fb8c2,1,1,1
6,2,10,10,discount,fafdcd668e3743c1bb461111dcafc2a4,1,1,1
7,0,0,3,informational,5a8bc65990b245e5a138643cd4eb9837,0,1,1
8,5,5,5,bogo,f19421c1d4aa40978ebb69ca19b0e20d,1,1,1
9,2,10,7,discount,2906b810c7d4411798c6938adc9daaa5,1,1,0


Use one-hot encoding for 'offer_type' column since there are only three different types of offers. 

In [6]:
portfolio = pd.concat([portfolio, pd.get_dummies(portfolio['offer_type']).add_prefix('type_')], axis=1)
portfolio.drop(columns=['offer_type'], inplace=True)
portfolio

,reward,difficulty,duration,id,channel_web,channel_mobile,channel_social,type_bogo,type_discount,type_informational
0,10,10,7,ae264e3637204a6fb9bb56bc8210ddfd,0,1,1,1,0,0
1,10,10,5,4d5c57ea9a6940dd891ad53e9dbe8da0,1,1,1,1,0,0
2,0,0,4,3f207df678b143eea3cee63160fa8bed,1,1,0,0,0,1
3,5,5,7,9b98b8c7a33c4b65b9aebfe6a799e6d9,1,1,0,1,0,0
4,5,20,10,0b1e1539f2cc45b7b9fa7c272da2e1d7,1,0,0,0,1,0
5,3,7,7,2298d6c36e964ae4a3e7e9706d1fb8c2,1,1,1,0,1,0
6,2,10,10,fafdcd668e3743c1bb461111dcafc2a4,1,1,1,0,1,0
7,0,0,3,5a8bc65990b245e5a138643cd4eb9837,0,1,1,0,0,1
8,5,5,5,f19421c1d4aa40978ebb69ca19b0e20d,1,1,1,1,0,0
9,2,10,7,2906b810c7d4411798c6938adc9daaa5,1,1,0,0,1,0


`portfolio` is ready to be joined into `response`. Make an encoded copy on the drive. 

In [7]:
portfolio.to_csv(os.path.join(data_dst, 'portfolio_encoded.csv'), index=False)

### profile

In [8]:
profile.head()

,gender,age,id,became_member_on,income,profile_nan
0,U,55.0,68be06ca386d4c31939f3a4f0e3dd783,20170212,64000.0,1
1,F,55.0,0610b486422d4921ae7d2bf64640c50b,20170715,112000.0,0
2,U,55.0,38fe809add3b4fcf9315a9694bb96ff5,20180712,64000.0,1
3,F,75.0,78afa995795e4d85b5d9ceeca43f5fef,20170509,100000.0,0
4,U,55.0,a03223e636434f42ac4c3df47e8bac43,20170804,64000.0,1


Transform the 'became_member_on' column to datetime.

In [9]:
profile['became_member_on'] = pd.to_datetime(profile['became_member_on'], format='%Y%m%d')
profile['became_member_on'].max()

Timestamp('2018-07-26 00:00:00')

The most recent member signed up on 2018-7-26. Since there is no datetime information available in the `transcript`, convert this column to the total time (in day) in membership as of a later date (2018-8-1). 

In [10]:
profile['days_as_member'] = (pd.to_datetime('2018-08-01') - profile['became_member_on']).dt.days
profile.drop(columns=['became_member_on'], inplace=True)
profile.head()

,gender,age,id,income,profile_nan,days_as_member
0,U,55.0,68be06ca386d4c31939f3a4f0e3dd783,64000.0,1,535
1,F,55.0,0610b486422d4921ae7d2bf64640c50b,112000.0,0,382
2,U,55.0,38fe809add3b4fcf9315a9694bb96ff5,64000.0,1,20
3,F,75.0,78afa995795e4d85b5d9ceeca43f5fef,100000.0,0,449
4,U,55.0,a03223e636434f42ac4c3df47e8bac43,64000.0,1,362


Use one-hot encoding for 'gender' column since there are only four different labels. 

In [11]:
profile = pd.concat([profile, pd.get_dummies(profile['gender']).add_prefix('gender_')], axis=1)
profile.drop(columns=['gender'], inplace=True)
profile.head()

,age,id,income,profile_nan,days_as_member,gender_F,gender_M,gender_O,gender_U
0,55.0,68be06ca386d4c31939f3a4f0e3dd783,64000.0,1,535,0,0,0,1
1,55.0,0610b486422d4921ae7d2bf64640c50b,112000.0,0,382,1,0,0,0
2,55.0,38fe809add3b4fcf9315a9694bb96ff5,64000.0,1,20,0,0,0,1
3,75.0,78afa995795e4d85b5d9ceeca43f5fef,100000.0,0,449,1,0,0,0
4,55.0,a03223e636434f42ac4c3df47e8bac43,64000.0,1,362,0,0,0,1


`profile` is ready to be joined into `response`. Make an encoded copy on the drive. 

In [12]:
profile.to_csv(os.path.join(data_dst, 'profile_encoded.csv'), index=False)

## Generate data for modeling

Join `portfolio` and `profile` with `response`. 

In [13]:
all_data = pd.merge(response, portfolio, left_on='offer', right_on='id')
all_data = pd.merge(all_data, profile, left_on='person', right_on='id')
all_data.drop(columns=['person', 'offer', 'id_x', 'id_y'], inplace=True)
all_data.head()

,positive_response,reward,difficulty,duration,channel_web,channel_mobile,channel_social,type_bogo,type_discount,type_informational,age,income,profile_nan,days_as_member,gender_F,gender_M,gender_O,gender_U
0,False,2,10,7,1,1,0,0,1,0,33.0,72000.0,0,467,0,1,0,0
1,False,0,0,4,1,1,0,0,0,1,33.0,72000.0,0,467,0,1,0,0
2,False,0,0,3,0,1,1,0,0,1,33.0,72000.0,0,467,0,1,0,0
3,False,5,5,5,1,1,1,1,0,0,33.0,72000.0,0,467,0,1,0,0
4,False,2,10,10,1,1,1,0,1,0,33.0,72000.0,0,467,0,1,0,0


In [14]:
all_data.to_csv(os.path.join(data_dst, 'all_data.csv'), index=False)

## Split training and test data

In [15]:
from sklearn.model_selection import train_test_split

y = all_data['positive_response'].astype(int)
X = all_data.drop(columns=['positive_response'])
train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=0.2, random_state=41)

In [16]:
train_X.to_csv(os.path.join(data_dst, 'train_X.csv'), index=False)
train_y.to_csv(os.path.join(data_dst, 'train_y.csv'), index=False)
test_X.to_csv(os.path.join(data_dst, 'test_X.csv'), index=False)
test_y.to_csv(os.path.join(data_dst, 'test_y.csv'), index=False)